# Rapport-notater: BERT (Extractive QA) + XLNet (klassifisering)

Strukturerte notater du kan bruke direkte i bachelor-/rapporttekst.

## Innhold

- Transformer: encoder vs decoder
- BERT QA: inputformat, start/end logits, softmax og span-valg
- XLNet klassifisering: preprocessing, label encoding, padding/attention_mask
- Lagring + inferens
- Kritisk refleksjon (metrikker, datasettstørrelse, truncation)


## 1) Transformer: encoder vs decoder

- **Encoder-only** (BERT/RoBERTa/DistilBERT): forståelse/representasjon
- **Decoder-only** (GPT): generering
- **Encoder–decoder** (T5/BART): seq2seq (oversettelse/summarization)

Huske-regel:

- Encoder = les/forstå
- Decoder = skriv


## 2) Pipeline A: BERT Question Answering (Extractive QA)

### 2.1 Inputformat

Vanlig format:

- `[CLS] question [SEP] context [SEP]`

Segmentering (`token_type_ids`) markerer ofte:

- 0 = spørsmål
- 1 = kontekst

### 2.2 Modelloutput

`BertForQuestionAnswering` gir:

- `start_logits`: score per token for start
- `end_logits`: score per token for slutt

### 2.3 Softmax (logits → sannsynlighet)

\[
\mathrm{softmax}(z_i) = \frac{e^{z_i}}{\sum_j e^{z_j}}
\]

I praksis kan sannsynlighet brukes som confidence-score.

### 2.4 Span-valg

En robust strategi velger (i, j) der j ≥ i og ofte med maks-lengde.


In [ ]:
# BERT QA (kode-mal) — krever transformers + torch

# from transformers import BertForQuestionAnswering, BertTokenizer
# import torch
#
# model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
# model = BertForQuestionAnswering.from_pretrained(model_name)
# tok = BertTokenizer.from_pretrained(model_name)
#
# question = "When was the first DVD released?"
# context = "The first DVD (Digital Versatile Disc) was released on March 24, 1997..."
#
# enc = tok.encode_plus(question, context, return_tensors="pt")
# with torch.no_grad():
#     out = model(**enc)
#
# start_idx = out.start_logits.argmax(dim=-1).item()
# end_idx = out.end_logits.argmax(dim=-1).item()
#
# input_ids = enc["input_ids"][0]
# tokens = tok.convert_ids_to_tokens(input_ids)
# answer = tok.convert_tokens_to_string(tokens[start_idx : end_idx + 1])
# print(answer)


## 3) Pipeline B: XLNet tekstklassifisering (emosjoner)

### 3.1 Supervised learning

Mål: lære mappingen **tekst → klasse**.

### 3.2 Preprocessing

Rensing (fjerne støy som emoji/mentions) kan stabilisere trening.

### 3.3 Klassebalanse

Hvis datasettet er ubalansert, kan modellen lære å alltid gjette majoritetsklassen.

### 3.4 Tokenisering og embeddings (presis formulering)

Riktig beskrivelse:

- Tokenizer lager `input_ids` + `attention_mask`
- Embeddings genereres inne i modellens embedding-lag når `input_ids` mates inn

### 3.5 Padding/truncation

- `padding="max_length"` + `max_length=128` gir mange `<pad>`
- `attention_mask` styrer hva modellen ignorerer

### 3.6 Resultat (tolkning)

Lav accuracy på veldig lite treningsdata kan bety at pipeline er korrekt, men datasettet er for lite.


## 4) Kritisk refleksjon (gir pluss i vurdering)

- **Datasettstørrelse**: 100 samples ≈ tilfeldig nivå for 4 klasser (~0.25)
- **Metrikker**: rapporter også F1 per klasse + confusion matrix
- **QA span**: `argmax` på start/end kan gi suboptimale spans → bruk best-span søk med j ≥ i
- **Tokenisering/padding**: dokumenter strategi og verifiser `attention_mask`
- **Truncation**: evaluer ASR/utility med og uten truncation (sikkerhetsrelevant)

### Rapportformulering (klar og kort)

> “We saved intermediate checkpoints and evaluated both clean utility and attack success under different tokenization and truncation settings to ensure reproducibility and robust security evaluation.”


In [ ]:
# Softmax i Python (samme som formelen i LaTeX)
import math
from typing import Sequence, List


def softmax(logits: Sequence[float]) -> List[float]:
    # Stabil softmax: trekk fra max-logit for å unngå overflow
    m = max(logits)
    exps = [math.exp(z - m) for z in logits]
    s = sum(exps)
    return [e / s for e in exps]


z = [1.2, 0.3, -2.0]
softmax(z), sum(softmax(z))
